# Рекуррентные нейросети

Построим простейшую нейросеть для посимвольной генерации текста

In [2]:
import pandas as pd  # для работы с данными
import time  # для оценки времени выполнения
import torch  # для работы с нейронными сетями
from torch import nn  # для создания слоев нейронной сети

# Определяем устройство (GPU или CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


## Загрузка данных

Будем работать с датасетом реплик из Симпсонов. Нам нужно извлечь предобработанные тексты и закодировать их числами

In [3]:
# Загрузка данных
df = pd.read_csv('simpsons_script_lines.csv')
df.head()

C:\Users\lazar\AppData\Local\Temp\ipykernel_132\3859997088.py:2: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('simpsons_script_lines.csv')


,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,9549,32,209,"Miss Hoover: No, actually, it was a little of ...",848000,True,464.0,3.0,Miss Hoover,Springfield Elementary School,"No, actually, it was a little of both. Sometim...",no actually it was a little of both sometimes ...,31
1,9550,32,210,Lisa Simpson: (NEAR TEARS) Where's Mr. Bergstrom?,856000,True,9.0,3.0,Lisa Simpson,Springfield Elementary School,Where's Mr. Bergstrom?,wheres mr bergstrom,3
2,9551,32,211,Miss Hoover: I don't know. Although I'd sure l...,856000,True,464.0,3.0,Miss Hoover,Springfield Elementary School,I don't know. Although I'd sure like to talk t...,i dont know although id sure like to talk to h...,22
3,9552,32,212,Lisa Simpson: That life is worth living.,864000,True,9.0,3.0,Lisa Simpson,Springfield Elementary School,That life is worth living.,that life is worth living,5
4,9553,32,213,Edna Krabappel-Flanders: The polls will be ope...,864000,True,40.0,3.0,Edna Krabappel-Flanders,Springfield Elementary School,The polls will be open from now until the end ...,the polls will be open from now until the end ...,33


In [4]:
# Преобразуем тексты в список
phrases = df['normalized_text'].tolist()  # колонка с предобработанными текстами
phrases[:10]

['no actually it was a little of both sometimes when a disease is in all the magazines and all the news shows its only natural that you think you have it',
 'wheres mr bergstrom',
 'i dont know although id sure like to talk to him he didnt touch my lesson plan what did he teach you',
 'that life is worth living',
 'the polls will be open from now until the end of recess now just in case any of you have decided to put any thought into this well have our final statements martin',
 'i dont think theres anything left to say',
 'bart',
 'victory party under the slide',
 nan,
 'mr bergstrom mr bergstrom']

In [5]:
# Преобразуем каждый текст в список символов
text = [[char for char in phrase] for phrase in phrases if isinstance(phrase, str)]

## Создаём массив с данными

Нужно

1. Разбить данные на токены (у нас символы)
2. Закодировать числами
3. Превратить в эмбеддинги

In [6]:
# Определяем допустимые символы
CHARS = set('abcdefghijklmnopqrstuvwxyz ')  # все символы, которые мы хотим использовать для кодировки
INDEX_TO_CHAR = ['none'] + list(CHARS)  # все неизвестные символы будут получать тег 'none'
CHAR_TO_INDEX = {char: idx for idx, char in enumerate(INDEX_TO_CHAR)}  # словарь для преобразования символа в индекс

In [7]:
len(INDEX_TO_CHAR)

28

In [8]:
# Ограничиваем максимальную длину текста
MAX_LEN = 50

# Создаем тензор для хранения индексов символов
X = torch.zeros((len(text), MAX_LEN), dtype=int).to(device)  # перемещаем на устройство

# Заполняем тензор индексами символов
for i, phrase in enumerate(text):
    for j, char in enumerate(phrase):
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(char, CHAR_TO_INDEX['none'])

In [9]:
X[0:5]

tensor([[ 8, 16, 25, 19, 10,  7, 11, 19, 24, 24, 15, 25,  6,  7, 25,  5, 19,  4,
         25, 19, 25, 24,  6,  7,  7, 24, 20, 25, 16, 27, 25, 13, 16,  7, 21, 25,
          4, 16, 18, 20,  7,  6, 18, 20,  4, 25,  5, 21, 20,  8],
        [ 5, 21, 20,  3, 20,  4, 25, 18,  3, 25, 13, 20,  3, 26,  4,  7,  3, 16,
         18,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 6, 25,  2, 16,  8,  7, 25, 12,  8, 16,  5, 25, 19, 24,  7, 21, 16, 11,
         26, 21, 25,  6,  2, 25,  4, 11,  3, 20, 25, 24,  6, 12, 20, 25,  7, 16,
         25,  7, 19, 24, 12, 25,  7, 16, 25, 21,  6, 18, 25, 21],
        [ 7, 21, 19,  7, 25, 24,  6, 27, 20, 25,  6,  4, 25,  5, 16,  3,  7, 21,
         25, 24,  6, 22,  6,  8, 26,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 7, 21, 20, 25,  9, 16, 24, 24,  4, 25,  5,  6, 24, 24, 25, 13, 20, 25,
       

## Embedding и RNN ячейки

Каждому токену мы хотим сопоставить не просто число, но вектор. Поэтому вектор текста нам нужно умножить на матрицу эмбеддингов, которая тоже будет учиться в процессе обучения нейросети. Для создания такой матрицы нам нужен слой `nn.Embedding`

In [10]:
X[0:5].shape

torch.Size([5, 50])

In [11]:
# Создаем слой для эмбеддингов
embeddings = nn.Embedding(len(INDEX_TO_CHAR), 28).to(device)  # перемещаем на устройство
t = embeddings(X[0:5])  # Пример эмбеддинга для первых 5 фраз
print(t.shape)

torch.Size([5, 50, 28])


In [12]:
t.shape, X[0:5].shape

(torch.Size([5, 50, 28]), torch.Size([5, 50]))

In [13]:
# Создаем RNN слой
rnn = nn.RNN(28, 128, batch_first=True).to(device)  # перемещаем на устройство
output, hidden_state = rnn(t)

# Размеры выходных данных
print(output.shape, hidden_state.shape)

torch.Size([5, 50, 128]) torch.Size([1, 5, 128])


## Реализация сети с RNN
3 слоя:
1. Embeding (30)
2. RNN (hidden_dim=128)
3. Полносвязный слой для предсказания буквы (28, то есть размер словаря)

In [14]:
# Определяем архитектуру нейронной сети
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.embedding = nn.Embedding(len(INDEX_TO_CHAR), 30).to(device)  # слой эмбеддинга
        self.rnn = nn.RNN(30, 128).to(device)  # RNN слой
        self.out = nn.Linear(128, len(INDEX_TO_CHAR)).to(device)  # выходной слой

    def forward(self, sentences, state=None):
        x = self.embedding(sentences)  # преобразуем индексы в эмбеддинги
        x, hidden_state = self.rnn(x)  # пропускаем через RNN
        return self.out(x)  # возвращаем выходной слой

In [15]:
# Создаем модель, функцию потерь и оптимизатор
model = Network().to(device)  # перемещаем модель на устройство
criterion = nn.CrossEntropyLoss().to(device)  # функция потерь для многоклассовой классификации
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

Обучение:

In [16]:
# Обучение модели
for epoch in range(20):
    start = time.time()
    train_loss = 0.0
    train_passed = 0

    for i in range(int(len(X) / 100)):
        # Берем батч из 100 элементов
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1].to(device)  # входные данные
        Y_batch = batch[:, 1:].flatten().to(device)  # целевые данные

        optimizer.zero_grad()  # обнуляем градиенты
        answers = model(X_batch)  # получаем предсказания
        answers = answers.view(-1, len(INDEX_TO_CHAR))  # изменяем форму для функции потерь
        loss = criterion(answers, Y_batch)  # вычисляем потери
        train_loss += loss.item()

        loss.backward()  # обратное распространение ошибки
        optimizer.step()  # обновляем веса
        train_passed += 1

    # Выводим статистику по эпохе
    print(f"Epoch {epoch}. Time: {time.time() - start:.3f}, Train loss: {train_loss / train_passed:.3f}")


Epoch 0. Time: 57.995, Train loss: 1.831
Epoch 1. Time: 54.678, Train loss: 1.729
Epoch 2. Time: 58.122, Train loss: 1.712
Epoch 3. Time: 61.115, Train loss: 1.703
Epoch 4. Time: 59.554, Train loss: 1.697
Epoch 5. Time: 56.450, Train loss: 1.692
Epoch 6. Time: 58.767, Train loss: 1.685
Epoch 7. Time: 54.431, Train loss: 1.681
Epoch 8. Time: 56.086, Train loss: 1.678
Epoch 9. Time: 61.733, Train loss: 1.675
Epoch 10. Time: 56.695, Train loss: 1.672
Epoch 11. Time: 54.101, Train loss: 1.670
Epoch 12. Time: 54.102, Train loss: 1.668
Epoch 13. Time: 53.346, Train loss: 1.667
Epoch 14. Time: 52.265, Train loss: 1.666
Epoch 15. Time: 58.017, Train loss: 1.665
Epoch 16. Time: 66.032, Train loss: 1.664
Epoch 17. Time: 82.395, Train loss: 1.663
Epoch 18. Time: 72.687, Train loss: 1.662
Epoch 19. Time: 54.346, Train loss: 1.662


## Генерация


- Сначала отправлем в модель буквы из предложения (прогревая состояние)
- Затем берём самую вероятную букву и добавляем её в предложение
- Повторяем пока не получим none (0)

In [17]:
CHAR_TO_INDEX['none']

0

In [18]:
# Функция для генерации предложения
def generate_sentence(word):
    model.eval()  # переключаем модель в режим оценки
    with torch.no_grad():  # отключаем вычисление градиентов
        sentence = list(word)
        sentence = [CHAR_TO_INDEX.get(char, 0) for char in sentence]  # преобразуем символы в индексы
        sentence_tensor = torch.tensor(sentence).unsqueeze(0).to(device)  # перемещаем на устройство
        answers = model(sentence_tensor)  # получаем предсказания
        _, indices = answers.topk(1)  # выбираем наиболее вероятные символы
        return ''.join([INDEX_TO_CHAR[idx.item()] for idx in indices.flatten()])  # преобразуем индексы обратно в символы


In [19]:
# Пример работы
test_text = "hello world"
print(f"Generated sentence: {generate_sentence(test_text)}")

Generated sentence: e ll te el 
